# Notebook for the experiments
In this notebook are contained the following features:
* GRAFF + Link prediction,

The main tools that have been exploited are [PyTorch](https://pytorch.org/) (1.13.0), [PyTorch-Lightning](https://www.pytorchlightning.ai/index.html) (1.5.10), [Pytorch-geometric](https://pytorch-geometric.readthedocs.io/en/latest/install/installation.html) (2.3.0) and [Weights & Biases](https://wandb.ai/)

### Requirements to run the notebook

In [1]:
# !pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
# !pip install pytorch-lightning==1.5.10
# !pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-1.12.0+cu113.html
# !pip install torch_geometric
# !pip install wandb
# !pip install ogb

## Importing the libraries

In [2]:
######## IMPORT EXTERNAL FILES ###########
import torch
import torch.nn.functional as F
import torch.nn.utils.parametrize as parametrize
import torch.nn as nn

import torch_geometric
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.utils import negative_sampling

import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.loggers import WandbLogger

import wandb
######### IMPORT INTERNAL FILES ###########
import sys
sys.path.append("../../src")
from GRAFF import *
from config import *

/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Link prediction features initialized.....


### System configuration

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_gpus = 1 if device == 'cuda' else 0

if wb:
    wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: difra00 (deepl_wizards). Use `wandb login --relogin` to force relogin


## PyTorch Lightning DataModule (Link Prediction)

In [4]:
class DataModuleLP(pl.LightningDataModule):

    def __init__(self,  train_set, val_set, test_set, mode, batch_size):

        self.mode = mode  # "hp" or "test"
        self.batch_size = batch_size
        self.train_set, self.val_set, self.test_set = train_set, val_set, test_set

    def setup(self, stage=None):
        if stage == 'fit':

            # edge_index are the message passing edges,
            # edge_label_index are the supervision edges.
            if self.train_set.pos_edge_label_index.shape[1] < self.train_set.edge_index.shape[1]:
                pos_mask_edge = self.train_set.pos_edge_label_index.shape[1]

                self.train_set.edge_index = self.train_set.edge_index[:, pos_mask_edge:]
            else:
                self.train_set.pos_edge_label_index = self.train_set.edge_index[:, :self.train_set.edge_index.shape[1] // 2]
                self.train_set.neg_edge_label_index = self.train_set.neg_edge_label_index[:, :self.train_set.edge_index.shape[1] // 2]

                self.train_set.edge_index = self.train_set.edge_index[:, self.train_set.edge_index.shape[1] // 2:]

                

    def train_dataloader(self, *args, **kwargs):
        return DataLoader([self.train_set], batch_size=batch_size, shuffle=False)

    def val_dataloader(self, *args, **kwargs):
        if self.mode == 'hp':
            return DataLoader([self.val_set], batch_size=batch_size, shuffle=False)
        elif self.mode == 'test':
            return DataLoader([self.test_set], batch_size=batch_size, shuffle=False)

In [5]:
train_data = torch.load(dataset_name + "/train_data.pt")
val_data = torch.load(dataset_name + "/val_data.pt")
test_data = torch.load(dataset_name + "/test_data.pt")

In [6]:
# print(train_data)
# print(val_data)
# print(test_data)


In [15]:
mode = 'test'  # hp: Hyperparameter selection mode
sweep = False
dataM = DataModuleLP(train_data.clone(), val_data.clone(), test_data.clone(), mode = mode, batch_size = batch_size)
dataM.setup(stage='fit')
dataM.setup(stage='test') 


In [16]:
print(dataM.train_set)
print(dataM.val_set)
print(dataM.test_set)


Data(x=[2708, 1433], edge_index=[2, 4224], y=[2708], train_mask=[2708, 10], val_mask=[2708, 10], test_mask=[2708, 10], pos_edge_label=[4224], pos_edge_label_index=[2, 4224], neg_edge_label=[844800], neg_edge_label_index=[2, 844800])
Data(x=[2708, 1433], edge_index=[2, 8448], y=[2708], train_mask=[2708, 10], val_mask=[2708, 10], test_mask=[2708, 10], pos_edge_label=[527], pos_edge_label_index=[2, 527], neg_edge_label=[105400], neg_edge_label_index=[2, 105400])
Data(x=[2708, 1433], edge_index=[2, 9502], y=[2708], train_mask=[2708, 10], val_mask=[2708, 10], test_mask=[2708, 10], pos_edge_label=[527], pos_edge_label_index=[2, 527], neg_edge_label=[105400], neg_edge_label_index=[2, 105400])


### PyTorch Lightning Callbacks

In [17]:

class Get_Metrics(Callback):

    def on_train_epoch_end(self, trainer: "pl.Trainer", pl_module: "pl.LightningModule"):

        # Compute the metrics
        train_loss = sum(
            pl_module.train_prop['loss']) / len(pl_module.train_prop['loss'])
        train_acc100 = sum(
            pl_module.train_prop['HR@100']) / len(pl_module.train_prop['HR@100'])
        # train_acc20 = sum(
        #     pl_module.train_prop['HR@20']) / len(pl_module.train_prop['HR@20'])
        # train_acc1 = sum(
        #     pl_module.train_prop['HR@1']) / len(pl_module.train_prop['HR@1'])
        test_loss = sum(
            pl_module.test_prop['loss']) / len(pl_module.test_prop['loss'])
        
        test_acc100 = sum(pl_module.test_prop['HR@100']) / \
            len(pl_module.test_prop['HR@100'])
        # test_acc20 = sum(pl_module.test_prop['HR@20']) / \
        #     len(pl_module.test_prop['HR@20'])
        # test_acc1 = sum(pl_module.test_prop['HR@1']) / \
        #     len(pl_module.test_prop['HR@1'])

        # Log the metrics
        pl_module.log(name='Loss on train', value=train_loss,
                      on_epoch=True, prog_bar=True, logger=True)
        pl_module.log(name='Loss on test', value=test_loss,
                      on_epoch=True, prog_bar=True, logger=True)

        pl_module.log(name='HR@100 on train', value=train_acc100,
                      on_epoch=True, prog_bar=True, logger=True)
        pl_module.log(name='HR@100 on test', value=test_acc100,
                            on_epoch=True, prog_bar=True, logger=True)
        
        # pl_module.log(name='HR@20 on train', value=train_acc20,
        #               on_epoch=True, prog_bar=True, logger=True)
        # pl_module.log(name='HR@20 on test', value=test_acc20,
        #               on_epoch=True, prog_bar=True, logger=True)
        
        
        # pl_module.log(name='HR@1 on train', value=train_acc1,
        #               on_epoch=True, prog_bar=True, logger=True)
        # pl_module.log(name='HR@1 on test', value=test_acc1,
        #               on_epoch=True, prog_bar=True, logger=True)

        
        
        
        

        # Re-initialize the metrics
        pl_module.train_prop['loss'] = []
        pl_module.train_prop['HR@100'] = []
        pl_module.train_prop['HR@20'] = []
        pl_module.train_prop['HR@1'] = []

        pl_module.test_prop['loss'] = []
        pl_module.test_prop['HR@100'] = []
        pl_module.test_prop['HR@20'] = []
        pl_module.test_prop['HR@1'] = []


## PyTorch Lightning Training Module (Node Classification)

In [18]:
class TrainingModule(pl.LightningModule):

    def __init__(self, model, predictor, lr, wd):
        super().__init__()
        self.model = model.to(device)
        self.predictor = predictor.to(device)
        self.lr = lr
        self.wd = wd

        self.train_prop = {'loss': [], 'HR@100': [], 'HR@20': [], 'HR@1': []}
        self.test_prop = {'loss': [], 'HR@100': [], 'HR@20': [], 'HR@1': []}

    def training_step(self, batch, batch_idx):

        out = self.model(batch)

        pos_edge = batch.pos_edge_label_index

        pos_pred = self.predictor(out[pos_edge[0]], out[pos_edge[1]], training = True)
        
        neg_edge = batch.neg_edge_label_index

        neg_pred = self.predictor(out[neg_edge[0]], out[neg_edge[1]], training = True)


        loss = -torch.log(pos_pred + 1e-15).mean() - \
            torch.log(1 - neg_pred[:pos_pred.shape[0]] + 1e-15).mean()
        
        acc100 = evaluate(pos_pred, neg_pred[pos_pred.shape[0]:], k = 100)
        # acc20 = evaluate(pos_pred, neg_pred, k = 20)
        # acc1 = evaluate(pos_pred, neg_pred, k = 1)


        self.train_prop['loss'].append(loss)
        self.train_prop['HR@100'].append(acc100)
        # self.train_prop['HR@20'].append(acc20)
        # self.train_prop['HR@1'].append(acc1)



        return loss

    def validation_step(self, batch, batch_idx):
        
        out = self.model(batch)

        pos_edge = batch.pos_edge_label_index

        pos_pred = self.predictor(out[pos_edge[0]], out[pos_edge[1]], training = False)  # training is for dropout
        
       
        neg_edge = batch.neg_edge_label_index

        neg_pred = self.predictor(out[neg_edge[0]], out[neg_edge[1]], training = False)  # training is for dropout

        

        loss = -torch.log(pos_pred + 1e-15).mean() - \
            torch.log(1 - neg_pred[:pos_pred.shape[0]] + 1e-15).mean()
        
        acc100 = evaluate(pos_pred, neg_pred[pos_pred.shape[0]:], k = 100)
        # acc20 = evaluate(pos_pred, neg_pred, k = 20)
        # acc1 = evaluate(pos_pred, neg_pred, k = 1)
        
        self.test_prop['loss'].append(loss)
        self.test_prop['HR@100'].append(acc100)
        # self.test_prop['HR@20'].append(acc20)
        # self.test_prop['HR@1'].append(acc1)

        return loss


    def configure_optimizers(self):
        self.optimizer = torch.optim.Adam(
            list(self.model.parameters()) + list(self.predictor.parameters()), lr=self.lr, weight_decay=self.wd)
        return self.optimizer

def evaluate(pos_pred, neg_pred, k = 100):

    n_indices = pos_pred.shape[0]
    hr = 0

    k = min(neg_pred.shape[0]+1, k)

    for pos_idx in range(n_indices):
        pos = pos_pred[pos_idx].unsqueeze(0)

        # print("pos_score is: ", pos)
        tot_tensor = torch.cat((pos, neg_pred), dim = 0)
        # print("tot_tensor is: ", tot_tensor.shape)

        scores_idx = torch.topk(tot_tensor.squeeze(1), k).indices

        if 0 in scores_idx:
            hr += 1
    return hr/n_indices

In [19]:

# hp enables a grid search on a wide set of hyperparameters.
if not sweep or mode == 'test':
    model = PhysicsGNN_LP(dataset, hidden_dim, num_layers, step = step)
    # model = GNNStack(dataset.x.shape[1], hidden_dim, hidden_dim, num_layers, dropout, emb=True)
    predictor = LinkPredictor(
        hidden_dim, output_dim, mlp_layer, link_bias, dropout, device=device)
    # predictor = LinkPredictor(
    #      hidden_dim, hidden_dim, 1, num_layers,
    #              dropout)
    
    pl_training_module = TrainingModule(model, predictor, lr, wd)

### Hyperparameters Tuning

In [20]:
def sweep_train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        model = PhysicsGNN_LP(dataset, config.hidden_dim,
                              config.num_layers, step=config.step)
        predictor = LinkPredictor(
            config.hidden_dim, config.output_dim, config.mlp_layer, config.link_bias, config.dropout, device=device)
        pl_training_module = TrainingModule(
            model, predictor, config.lr, config.wd)
        exp_name = "Sweep_LinkPred"
        wandb_logger = WandbLogger(
            project=project_name, name=exp_name, config=hyperparameters)
        trainer = trainer = pl.Trainer(
            max_epochs=epochs,  # maximum number of epochs.
            gpus=num_gpus,  # the number of gpus we have at our disposal.
            default_root_dir="", callbacks=[Get_Metrics(), EarlyStopping('Loss on test', mode='min', patience=15)],
            logger=wandb_logger
        )
        trainer.fit(model=pl_training_module, datamodule=dataM)


if mode == 'hp' and sweep:

    import pprint

    pprint.pprint(sweep_config)

    sweep_id = wandb.sweep(sweep_config, project=project_name)

    wandb.agent(sweep_id, sweep_train, count=500)

    wandb.finish()

In [21]:


if wb:
    exp_name = "Node_class_lr: " + \
        str(hyperparameters['learning rate']) + \
        '_wd: ' + str(hyperparameters['weight decay'])
    description = ' initial tests'
    exp_name += description
    wandb_logger = WandbLogger(
        project=project_name, name=exp_name, config=hyperparameters)


trainer = trainer = pl.Trainer(
    max_epochs=epochs,  # maximum number of epochs.
    gpus=num_gpus,  # the number of gpus we have at our disposal.
    default_root_dir="", callbacks=[Get_Metrics(), EarlyStopping('Loss on test', mode='min', patience=15)],
    logger=wandb_logger if wb else None

)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [22]:
trainer.fit(model = pl_training_module, datamodule = dataM)
if wb:
    wandb.finish()

/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type          | Params
--------------------------------------------
0 | model     | PhysicsGNN_LP | 2.5 M 
1 | predictor | LinkPredictor | 34.8 K
--------------------------------------------
2.6 M     Trainable params
0         Non-trainable params
2.6 M     Total params
10.216    Total estimated model params size (MB)
/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /home/peppe/Desktop/Università/Projects/Link_Prediction_with_Physics-based_GNNs-/src/link_prediction/Node_class_lr: 0.01_wd: 0.01 initial tests/version_None/checkpoints e

/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/peppe/anaconda3/envs/my_env/lib/python3.10/site-packages/pytorch_lightning/trainer/data_loading.py:432: UserWarning: The number of training samples (1) is smaller than the logging interval Trainer(log_every_n_s

Epoch 0: 100%|██████████| 2/2 [00:01<00:00,  1.84it/s, loss=1.39, Loss on train=1.390, Loss on test=1.390, HR@100 on train=0.000237, HR@100 on test=0.0019]

Epoch 16: 100%|██████████| 2/2 [00:01<00:00,  1.89it/s, loss=1.39, v_num=dfcu, Loss on train=1.390, Loss on test=1.390, HR@100 on train=0.0298, HR@100 on test=1.000]   


HR@100 on test,▁▁▁▁▁▁▁▁▁▁▁▂▂▃▄▇█
HR@100 on train,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁█
Loss on test,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Loss on train,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▂▂▃▃▄▄▅▅▅▆▆▇▇██
trainer/global_step,▁▁▂▂▃▃▄▄▅▅▅▆▆▇▇██
HR@100 on test,1.0
HR@100 on train,0.02983
Loss on test,1.38629
Loss on train,1.38629
epoch,16
